In [2]:
# Import the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from IPython.display import Image
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix, roc_auc_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
#Functions
def slice_by_country(country_names, country_column_name, data_frame):
    sliced_df = data_frame[data_frame[country_column_name].isin(country_names)]
    return sliced_df
def map_countries_to_numeric(countries_mapping,country_column_name,numeric_column_name,df):
    numeric_mapping_vals = []
    # loop through the rows using iterrows()
    for index, row in df.iterrows():
        if row[country_column_name] in countries_mapping:
            numeric_mapping_vals += [countries_mapping[row[country_column_name]]]
            print(numeric_mapping_vals)
    df[numeric_column_name] = numeric_mapping_vals
    return df

# Function to make predictions for a country
def predict_country(country_data, future_years, meat_consumption_category):
    years = country_data['Year'].values.reshape(-1, 1)
    
    # Population prediction
    pop_model = LinearRegression()
    pop_model.fit(years, country_data['population_historical'])
    future_population = pop_model.predict(future_years)
    
    # Consumption prediction
    cons_model = LinearRegression()
    cons_model.fit(years, country_data[meat_consumption_category])
    future_consumption = cons_model.predict(future_years)
    
    return future_population, future_consumption

# Generate future years
future_years = np.array(range(2025, 2032)).reshape(-1, 1)

population_df = pd.read_csv("https://ourworldindata.org/grapher/population.csv?country=USA~BRA~AUS~ESP~ZWE~MDV~JPN&v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
meat_df = pd.read_csv("Consumption of meat per capita.csv")
combined_df = pd.merge(population_df, meat_df, on=["Entity", "Year"], how="inner")
combined_df.head()

# Create plots for each country
countries = combined_df['Entity'].unique()
#display(countries)

# Store predictions for printing
predictions = []

for idx, country in enumerate(countries):
    country_data = combined_df[combined_df['Entity'] == country]
    future_pop, future_cons = predict_country(country_data, future_years, 'Poultry')
    
    # Store predictions
    for year, pop, cons in zip(future_years.flatten(), future_pop, future_cons):
        predictions.append({
            'Country': country,
            'Year': int(year),
            'Population': round(pop, 1),
            'Poultry_Consumption': round(cons, 1)
        })
        
# Print predictions in a formatted table
predictions_df = pd.DataFrame(predictions)
print("\nPredictions for 2025-2031:")
print("\nCountry | Year | Population (millions) | Poultry Consumption (kg per capita)")
print("-" * 75)
for _, row in predictions_df.sort_values(['Country', 'Year']).iterrows():
    print(f"{row['Country']:<7} | {row['Year']} | {row['Population']:>18.1f} | {row['Poultry_Consumption']:>31.1f}")

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cuba', 'Cyprus', 'Czechia', 'Czechoslovakia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia',
       'Ethiopia (former)', 'Fiji', 'Finland', 'France',
       'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Greece', 'Grenada', '

In [ ]:
population_df = pd.read_csv("https://ourworldindata.org/grapher/population.csv?country=USA~BRA~AUS~ESP~ZWE~MDV~JPN&v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
meat_df = pd.read_csv("Consumption of meat per capita.csv")

In [ ]:
def filter_countries_by_names(data_frame, countries_list, country_column_name):
    filtered_df = data_frame[data_frame[country_column_name].isin(countries_list)]
    return filtered_df
countries_to_pull = ['United States', 'Brazil', 'Australia', 'Spain', 'Zimbabwe', 'Maldives', 'Japan']
filtered_population_df = filter_countries_by_names(population_df, countries_to_pull, 'Entity')
filtered_population_df = filtered_population_df.drop(columns=['Code'])
print(filtered_population_df)

In [ ]:
years_to_include = list(range(1961, 2022))
filtered_population_df_years = filtered_population_df[filtered_population_df['Year'].isin(years_to_include)]
filtered_population_df_years

In [ ]:
meat_df = filter_countries_by_names(meat_df, countries_to_pull, 'Entity')
meat_df = meat_df.drop(columns= ['Beef', 'Sheep and goat', 'Pork', 'Other meats'])
meat_df.head()

In [ ]:
filtered_population_df_years["Year"] = pd.to_numeric(filtered_population_df_years["Year"], errors='coerce')
meat_df["Year"] = pd.to_numeric(meat_df["Year"], errors='coerce')
combined_df = pd.merge(filtered_population_df_years, meat_df, on=["Entity", "Year"], how="inner")
combined_df.head()

In [ ]:
combined_df = slice_by_country(['United States', 'Brazil', 'Austrailia', 'Spain', 'Zimbabwe', 'Maldives', 'Japan'],'Entity',combined_df)
combined_df = map_countries_to_numeric({'United States':1, 'Brazil':2, 'Austrailia':3, 'Spain':4, 'Zimbabwe':5, 'Maldives':6, 'Japan':7},'Entity','country_index',combined_df)
